In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torchvision import datasets
from torchvision.transforms import transforms

from lib.path import data_file_path, model_file_path, output_path


In [3]:
data_transform = transforms.Compose(
    # fmt : off
    [
        transforms.ToTensor(),
        transforms.Resize(32),
        transforms.Normalize((0.5,), (1.0,)),  # 간이 정규화
    ]
    # fmt : on
)

train_data = datasets.MNIST(
    root=data_file_path(), train=True, download=True, transform=data_transform
)
test_data = datasets.MNIST(
    root=data_file_path(), train=False, download=True, transform=data_transform
)


In [4]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)


In [5]:
next(iter(train_loader))[0].shape

torch.Size([32, 1, 32, 32])

In [6]:
class Lenet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)
        self.conv3 = nn.Conv2d(
            in_channels=16, out_channels=120, kernel_size=5, stride=1
        )
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.tanh(x)
        x = F.max_pool2d(x, 2, 2)
        x = self.conv2(x)
        x = F.tanh(x)
        x = F.max_pool2d(x, 2, 2)
        x = self.conv3(x)
        x = F.tanh(x)
        x = x.view(-1, 120)
        x = self.fc1(x)
        x = F.tanh(x)
        x = self.fc2(x)
        x = F.tanh(x)

        return x


model = Lenet()
model

Lenet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)

In [7]:
list(model.parameters())

[Parameter containing:
 tensor([[[[-0.1595,  0.1329, -0.1775,  0.0105,  0.1050],
           [-0.0575, -0.0254, -0.0836, -0.1889,  0.1688],
           [-0.1768,  0.0831, -0.1090,  0.1444, -0.0359],
           [-0.1905,  0.1175,  0.0085,  0.1085,  0.1371],
           [-0.1263, -0.0028,  0.0952,  0.1813, -0.1273]]],
 
 
         [[[ 0.0247, -0.0208,  0.1682,  0.0476, -0.0380],
           [-0.0405, -0.1978,  0.1914,  0.0373, -0.0050],
           [ 0.0305,  0.0803, -0.0351, -0.0800, -0.1041],
           [-0.0393,  0.1898, -0.0817,  0.1843, -0.1235],
           [ 0.0043,  0.0779,  0.1069, -0.1669,  0.1558]]],
 
 
         [[[ 0.1263, -0.0656,  0.0245, -0.1961, -0.0562],
           [-0.0111,  0.0795, -0.1670,  0.1450,  0.0921],
           [ 0.1507, -0.0543, -0.0596, -0.0207,  0.1818],
           [-0.1038,  0.0988, -0.0908,  0.1748,  0.1491],
           [ 0.0809,  0.0643,  0.1204,  0.1609,  0.0027]]],
 
 
         [[[-0.0741, -0.1257, -0.0634,  0.1024, -0.1067],
           [-0.1402,  0.0266,  

In [8]:
model(torch.randn(1, 1, 32, 32))

tensor([[-0.1348, -0.0759,  0.0406,  0.0207,  0.0445, -0.1271, -0.0637,  0.0245,
         -0.1488,  0.1153]], grad_fn=<TanhBackward0>)

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

Lenet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)

In [10]:
from torchsummary import summary

summary(model, input_size=(1, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
            Conv2d-2           [-1, 16, 10, 10]           2,416
            Conv2d-3            [-1, 120, 1, 1]          48,120
            Linear-4                   [-1, 84]          10,164
            Linear-5                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.24
Estimated Total Size (MB): 0.29
----------------------------------------------------------------


In [11]:
from lib.train import TrainModel

train_model = TrainModel(model, train_loader)
train_model.train()

epoch : 2 loss : 0.83: 100%|██████████| 1875/1875 [00:24<00:00, 75.78it/s]


AttributeError: 'TrainModel' object has no attribute 'optim'

In [ ]:
import tqdm
from torch.utils.tensorboard import SummaryWriter

log_dir = output_path() / 'tensorboard'
writer = SummaryWriter(log_dir=log_dir)

optim = Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
epochs = 20

count = 0

# 체크포인트 불러오기


for ep in range(epochs):
    train_tqdm = tqdm.tqdm(train_loader)
    for data, label in train_tqdm:
        data = data.to(device)
        label = label.to(device)
        optim.zero_grad()
        pred = model(data)
        loss = criterion(pred, label)
        writer.add_scalar('Loss/train', loss, count)
        count += 1
        loss.backward()
        optim.step()

        train_tqdm.set_description(f'epoch : {ep + 1} loss : {loss.item():.2f}')
    # 체크포인트 저장

In [ ]:
model.eval()

with torch.no_grad():
    total_corr = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        preds = model(images)

        _, pred = torch.max(preds, 1)
        total_corr += (pred == labels).sum().item()

print(f'Acc {total_corr / len(test_data.targets)}')

In [ ]:
labels

In [ ]:
# 가중치만 저장
torch.save(model.state_dict(), model_file_path('lenet_cnn.pth'))

In [ ]:
model_new = Lenet()
model_new.load_state_dict(torch.load(model_file_path('lenet_cnn.pth')))

In [ ]:
# 모델을 통째로 저장
torch.save(model, model_file_path('lenet_model.pth'))

In [ ]:
# 체크 포인트 저장
torch.save(
    {
        'epoch': ep,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optim.state_dict(),
        'loss': loss,
    },
    model_file_path('checkpoint.pth'),
)

checkpoint = torch.load('checkpoint.pth')

model.load_state_dict(checkpoint['model_state_dict'])
optim.load_state_dict(checkpoint['optimizer_state_dict'])
ep = checkpoint['epoch']
loss = checkpoint['loss']

NameError: name 'torch' is not defined

In [ ]:
# 추론(infer)

from PIL import Image

img = Image.open(data_file_path('4.png'))
img.size

In [ ]:
import numpy as np

img_np = np.array(img)
img_np.shape

In [ ]:
# 학습할때와 같은 전처리
infer_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Grayscale(),
        transforms.Resize(32),
        transforms.Normalize((0.5,), (1.0,)),  # 간이 정규화
    ]
)

# img = infer_transform(img_np)
img = infer_transform(img)
img.size()


In [ ]:
pred = model(img.to(device))
torch.max(pred, 1)[1].item()